In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [72]:
df=pd.read_csv('train.csv')

#Ostateczna obróbka w wersji prealpha
for i in range(len(df.columns)):
    print(df.columns[i], df[df.columns[i]].dtypes, df[df.iloc[:,i].isna()].shape[0])
    if (df[df.columns[i]].dtypes=='object'):
        df.loc[df.iloc[:,i].isna(),df.columns[i]]='Nothingface'
    else:
        df.loc[df.iloc[:,i].isna(),df.columns[i]]=1000
    print(df.columns[i], df[df.columns[i]].dtypes, df[df.iloc[:,i].isna()].shape[0])
df=df.loc[:,df.columns[(df.columns!='Name') & (df.columns!='Ticket') & (df.columns!='Cabin')]]

PassengerId int64 0
PassengerId int64 0
Survived int64 0
Survived int64 0
Pclass int64 0
Pclass int64 0
Name object 0
Name object 0
Sex object 0
Sex object 0
Age float64 177
Age float64 0
SibSp int64 0
SibSp int64 0
Parch int64 0
Parch int64 0
Ticket object 0
Ticket object 0
Fare float64 0
Fare float64 0
Cabin object 687
Cabin object 0
Embarked object 2
Embarked object 0


In [73]:
def proceed(p):
    partalles=0
    gini=1
    partalles=sum(p)
    for jj in range(classes):
        gini-=(p[jj]*p[jj])/(partalles*partalles)
    return (partalles, gini)

def purify(p):
    for jj in range(classes):
        p[jj]=0


i=0

X=[]
Dat=[df]
value=[]

Pred=1
classes=2

while (i<len(Dat) and i<1000):
    #Wartość Nodesa
    ZZ=Dat[i].groupby([Dat[i].columns[Pred]]).count()[Dat[i].columns[0]]
    ZZ=ZZ*ZZ
    value.append(1-sum(ZZ)/(len(Dat[i])*len(Dat[i])))
    #print(value[i])
    
    ij=0; vv=0
    p=[0]*classes
    allp=[0]*classes
    #print('Paar', i, len(Dat[i]))
    
    bjaest=(1,)
    #Wybór najlepszego punktu bicia kategorycznego
    for j in range(len(Dat[i].columns)):
        if (j==Pred):
            continue
        #Liczenie funkcji kosztu
        L=Dat[i].iloc[:,[j,Pred]].copy()
        L.sort_values(by=L.columns[0], inplace=True)
                
        wynne=0
        if(L.iloc[:,0].dtypes=='object'):
            for ii in range(len(L)):
                if (ii!=0 and L.iat[ii,0]!=L.iat[ii-1,0]):
                    partalles, gini=proceed(p)
                    purify(p)
                    wynne=wynne+partalles*gini
                p[L.iat[ii,1]]+=1
            partalles, gini=proceed(p)
            purify(p)
            
            wynne=wynne+partalles*gini
            
            wynne=wynne/len(L)
        
        #Miara bicia liczbowego
        else:
            wynne=10
            for ii in range(len(L)):
                allp[L.iat[ii,1]]+=1
            
            pointofsplit=-1
            for ii in range(len(L)):
                if (ii>0 and L.iat[ii,0]!=L.iat[ii-1,0]):
                    part1, gini1=proceed(p)
                    part2, gini2=proceed(list(pd.Series(allp)-pd.Series(p)))
                    
                    if (wynne>(part1*gini1+part2*gini2)/len(L)):
                        wynne=(part1*gini1+part2*gini2)/len(L)
                        pointofsplit=L.iat[ii,0]
                
                p[L.iat[ii,1]]+=1
                
            purify(p)
            purify(allp)
            
        
        #Znalezienie najlepszego bicia
        if (wynne<bjaest[0]):
            if (L.iloc[:,0].dtypes=='object'):
                bjaest=(wynne, L.columns[0])
            else:
                bjaest=(wynne, L.columns[0], pointofsplit)
    
    
    #Modyfikacja Drzewa
    if (bjaest[0]<value[i]):
        Dat[i].sort_values(by=bjaest[1])
        #Podział - zmienna kategoryczna
        if (len(bjaest)==2):
            Z=np.unique(Dat[i][bjaest[1]])
            X.append((0, bjaest[1], None, bjaest[0], len(Dat), len(Dat)+len(Z)))
            Dat[i].sort_values(by=bjaest[1], inplace=True)
            
            mil=Dat[i].columns.get_loc(bjaest[1])
            lastcut=0
            for j in range(1, len(Dat[i]), 1):
                if (Dat[i].iat[j,mil]!=Dat[i].iat[j-1,mil]):
                    Dat.append(Dat[i].iloc[lastcut:j,:].copy())
                    lastcut=j
                    
            Dat.append(Dat[i].iloc[lastcut:,:].copy())
            
        #Podział - zmienna numeryczna
        else:
            X.append((0, bjaest[1], bjaest[2], bjaest[0], len(Dat), len(Dat)+2))
            Dat.append(Dat[i][Dat[i][bjaest[1]]<bjaest[2]].copy())
            Dat.append(Dat[i][Dat[i][bjaest[1]]>=bjaest[2]].copy())
            
            
    else:
        p=[0]*classes
        maks=0
        for j in range(len(Dat[i])):
            s=Dat[i].iat[j,Pred]
            p[Dat[i].iat[j,Pred]]+=1
            if (p[s]>p[maks]):
                maks=s
        
        X.append((1, maks))
    
    i+=1

In [74]:
for i in range(len(X)):
    print(i, len(Dat[i]), X[i])

0 891 (0, 'Sex', None, 0.33336500038859046, 1, 3)
1 314 (0, 'Pclass', 3, 0.2835893593106032, 3, 5)
2 577 (0, 'Age', 7.0, 0.28663150318153285, 5, 7)
3 170 (0, 'Age', 3.0, 0.09551820728291308, 7, 9)
4 144 (0, 'Fare', 23.45, 0.4301994301994302, 9, 11)
5 24 (0, 'SibSp', 3, 0.07407407407407408, 11, 13)
6 553 (0, 'Pclass', 2, 0.25973958268079356, 13, 15)
7 2 (0, 'PassengerId', 531, 0.0, 15, 17)
8 168 (0, 'Fare', 29.0, 0.08666373883765185, 17, 19)
9 117 (0, 'Embarked', None, 0.4598103747238856, 19, 22)
10 27 (0, 'PassengerId', 120, 0.12037037037037036, 22, 24)
11 15 (1, 1)
12 9 (0, 'PassengerId', 262, 0.16666666666666666, 24, 26)
13 120 (0, 'PassengerId', 573, 0.4266647081864473, 26, 28)
14 433 (0, 'Fare', 56.4958, 0.2012996992706066, 28, 30)
15 1 (1, 0)
16 1 (1, 1)
17 69 (0, 'Fare', 28.7125, 0.15856777493606142, 30, 32)
18 99 (0, 'Parch', 2, 0.01885521885521886, 32, 34)
19 23 (0, 'Fare', 15.7417, 0.34782608695652173, 34, 36)
20 31 (0, 'PassengerId', 502, 0.2492925863044709, 36, 38)
21 63 (0,

In [77]:
print(df[['Cabin', 'Survived']].sort_values(by='Cabin'))

           Cabin  Survived
583          A10         0
475          A14         0
556          A16         1
284          A19         0
599          A20         1
630          A23         1
867          A24         0
647          A26         1
209          A31         1
185          A32         0
445          A34         1
806          A36         0
96            A5         0
23            A6         1
174           A7         0
737         B101         1
815         B102         0
523          B18         1
329          B18         1
170          B19         0
781          B20         1
690          B20         1
540          B22         1
745          B22         0
829          B28         1
61           B28         1
779           B3         1
54           B30         0
369          B35         1
641          B35         1
..           ...       ...
373  Nothingface         0
374  Nothingface         0
375  Nothingface         1
363  Nothingface         0
349  Nothingface         0
3

In [82]:
kgbo=[0.1,0.2,0.3]
kgb=[0.9,0.6,0.7]
print(kgb-kgbo)

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [112]:
kappa=(1, )
print(kappa[1])

IndexError: tuple index out of range

In [16]:
print(df)
#print(df.iloc[0:2,:])

     PassengerId  Survived  Pclass     Sex     Age  SibSp  Parch      Fare  \
383          384         1       1  female    35.0      1      0   52.0000   
272          273         1       2  female    41.0      0      1   19.5000   
888          889         0       3  female  1000.0      1      2   23.4500   
887          888         1       1  female    19.0      0      0   30.0000   
381          382         1       3  female     1.0      0      2   15.7417   
885          886         0       3  female    39.0      0      5   29.1250   
9             10         1       2  female    14.0      1      0   30.0708   
542          543         0       3  female    11.0      4      2   31.2750   
380          381         1       1  female    42.0      0      0  227.5250   
2              3         1       3  female    26.0      0      0    7.9250   
654          655         0       3  female    18.0      0      0    6.7500   
14            15         0       3  female    14.0      0      0